# Accessing the *All of Us* Data

*We recommend that all new users read this notebook to learn the basics of accessing the All of Us dataset*

Below you will find recommendedinstructions for accessing the *All of Us* Curated Data Repository (CDR) data for analysis. 

This notebook was run using a standard JupyterLab Compute Engine VM with the default configuration.  

## Tutorial's Objectives

**What should you expect?** This notebook will give you an overview of what data is available in and how to access and analyze the data in Verily Workbench. 

This tutorial is divided into the following sections:
1. **Data Overview:** *All of Us* CDR Registered Tier Version 8 overview 
2. **Setup:** How to set up this notebook, install and import software packages, and create a variable for your dataset.
3. **Data Access & Visualization:** Examples to access & visualize the data via queries
6. **Saving data & visualizations**: How to save data & visualizations to your workspace bucket
7. **Provenance**: How to save metadata on this notebook.


## Setup
<a id='read-from-bq'></a>

Before we begin, we will first install a few packages we will use in our analysis:

In [ ]:
# %pip install polars

In [ ]:
from google.cloud import bigquery
import pandas as pd
import polars as pl
import os
import subprocess
from pathlib import Path
import pandas_gbq
import matplotlib.pyplot as plt
import seaborn as sns
import psutil

In [ ]:
def print_resource_usage():
  """Monitor memory and CPU usage"""
  memory = psutil.virtual_memory()
  cpu_percent = psutil.cpu_percent(interval=1)

  print(f"Memory: {memory.used / 1e9:.1f}GB / {memory.total / 1e9:.1f}GB "
        f"({memory.percent:.1f}% used)")
  print(f"CPU: {cpu_percent:.1f}% used across {os.cpu_count()} cores")

  # Recommendations
  if memory.percent > 85:
      print("HIGH MEMORY - Consider upgrading to highmem or downsizing data")
  elif memory.percent < 40 and cpu_percent < 40:
      print("LOW USAGE - Could downsize to save costs")
  else:
      print("Good utilization")

In [ ]:
# This line allows for the plots to be displayed inline in the Jupyter notebook
%matplotlib inline

sns.set(style="ticks",font_scale=1)

In [ ]:
pl.Config.set_fmt_str_lengths(128)

# Set the row limit to a higher value
pl.Config.set_tbl_rows(50)

# show all columns in pandas
pd.set_option("display.max_columns", None)

# show full column width
pd.set_option('display.max_colwidth', 100)

## Function

In [ ]:
def polars_gbq(query):
    """Execute BigQuery SQL and return polars DataFrame"""
    client = bigquery.Client()
    return pl.from_arrow(client.query(query).result().to_arrow())

## Data Access

In the first notebook, we discussed how to create a variable to easily reference the CDR data in our analysis. However, for our purposes, we have already generated the variables in a notebook and will source this notebook to automatically obtain the variables we need below. Assuming you have run this notebook `/home/jupyter/workspace/aou-tutorial-notebooks/I. Getting Started with Verily Workbench/01_1. Getting Started with Workbench (Python).ipynb` or this one, `/home/jupyter/workspace/aou-tutorial-notebooks/Setting_Env_Variables.ipynb` already.

And then you can run the cell below to load the variables.


In [ ]:
%run /home/jupyter/workspace/test-notebooks/Setting_Env_Variables_p2.ipynb

In [ ]:
temp_bucket = os.environ['WORKSPACE_TEMP_BUCKET']
temp_bucket

In [ ]:
dataset = os.environ.get("WORKSPACE_CDR")
print(dataset)

<div class="alert alert-block alert-info">
Verily Workbench will be updated to include environment variables soon, which will let researchers access the CDR using one environment variable from any notebook. All notebook will be updated once environment variables are available. For now, we recommend you set the dataset variable in each notebook you create using the code provided above.
</blockquote></div>   

## Sample Queries & Visualizations
<a id='read-from-bq'></a>

There are multiple ways to interact with your data from a Verily Workbench cloud environment (e.g. JupyterLab). This notebook provides examples of how you may interact with your data using the [`pandas-gbq` library](https://googleapis.dev/python/pandas-gbq/latest/index.html). You can also access the data using [IPython magics for BigQuery](https://cloud.google.com/python/docs/reference/bigquery/latest/magics)

First, we can view and create a dataframe of all of the tables available in our dataset using the following query:

In [ ]:
query= f"""
SELECT
  table_name,
  table_type,
  creation_time
FROM
    `{dataset}`.INFORMATION_SCHEMA.TABLES
ORDER BY
  table_name
"""

In [ ]:
df=polars_gbq(query)
df.head()

We can make a CSV of all of the tables to reference later if needed, this will be saved to our bucket:

In [ ]:
# Make DF a CSV and save in bucket 
new_id = os.environ.get("WORKSPACE_CDR", "unknown_dataset")
# Extract the dataset name by splitting on the '.' character
dataset_name = new_id.split('.')[-1]  # e.g., "R2024Q3R8"

In [ ]:
# Dynamically create the output file name
output_file = f"{dataset_name}_CT_tables.csv"
print(output_file)

In [ ]:
# Save DataFrame to CSV
df.write_csv(output_file)

### Sample Device Data Visualization

The example below shows creating a query & visualizing data from the activity table. First we create a query & a dataframe of all columns from activity summary 

In [ ]:
activity_query= f"""
    SELECT * from `{dataset}`.activity_summary LIMIT 1000
"""

In [ ]:
activity_df=polars_gbq(activity_query)
activity_df.head()

We can create a scatterplot of steps vs calories

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(activity_df['steps'], activity_df['activity_calories'], alpha=0.5)
plt.title('Steps vs. Activity Calories')
plt.xlabel('Steps')
plt.ylabel('Activity Calories')
plt.grid(True)
plt.show()

Create a distribution of sedentary vs active minutes

In [ ]:
plt.figure(figsize=(8, 6))

# Sedentary minutes (drop nulls)
sedentary = activity_df['sedentary_minutes'].drop_nulls()
plt.hist(sedentary, bins=30, alpha=0.5, label='Sedentary Minutes')

# Total active minutes (drop nulls)
total_active = (
    activity_df['fairly_active_minutes'] +
    activity_df['lightly_active_minutes'] +
    activity_df['very_active_minutes']
).drop_nulls()
plt.hist(total_active, bins=30, alpha=0.5, label='Total Active Minutes')

plt.legend()
plt.title('Distribution of Sedentary vs. Active Minutes')
plt.xlabel('Minutes')
plt.ylabel('Frequency')
plt.show()

## Extract All of Us CDR Schema Information

Extract schema information for reference when developing queries and analyses.

In [ ]:
# Extract all schema information
schema_data = {}

In [ ]:
table_query = f"""
    SELECT
        table_name,
        column_name,
        ordinal_position,
        data_type,
        is_nullable
    FROM `{dataset}.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name IN (
        'person',
        'death',
        'condition_occurrence',
        'drug_exposure',
        'measurement',
        'observation',
        'visit_occurrence',
        'procedure_occurrence',
        'device_exposure',
        'concept',
        'concept_relationship',
        'concept_ancestor',
        'cb_criteria',
        'cb_criteria_ancestor',
        'cb_search_person',
        'cb_search_all_events'
    )
    ORDER BY table_name, ordinal_position
    """

schema_data['table_schemas'] = polars_gbq(table_query)

In [ ]:
schema_data['table_schemas'].head()

In [ ]:
counts = []
for table in schema_data['table_schemas'].unique('table_name')['table_name'].to_list():
    try:
        query = f"SELECT COUNT(*) as row_count FROM `{dataset}.{table}`"
        result = polars_gbq(query)
        count = result['row_count'][0]
        counts.append({'table_name': table, 'row_count': count})
        print(f"  {table}: {count:,} rows")
    except Exception as e:
        print(f"  {table}: Error - {e}")
        counts.append({'table_name': table, 'row_count': None})

schema_data['table_row_counts'] = pl.DataFrame(counts)

In [ ]:
vocab_query = f"""
    SELECT
        vocabulary_id,
        domain_id,
        COUNT(*) AS concept_count,
        COUNT(DISTINCT concept_class_id) AS class_count,
        COUNT(DISTINCT CASE WHEN standard_concept = 'S' THEN concept_id END) AS standard_concept_count,
        COUNT(DISTINCT CASE WHEN invalid_reason IS NULL THEN concept_id END) AS valid_concept_count
    FROM `{dataset}.concept`
    GROUP BY vocabulary_id, domain_id
    ORDER BY concept_count DESC
    """

schema_data['vocabulary_structure'] = polars_gbq(vocab_query)

In [ ]:
schema_data['vocabulary_structure'] = schema_data['vocabulary_structure'].with_columns([
    pl.when(pl.col('standard_concept_count').is_between(1, 19))
      .then(-9)
      .otherwise(pl.col('standard_concept_count'))
      .alias('standard_concept_count'),
    
    pl.when(pl.col('valid_concept_count').is_between(1, 19))
      .then(-9)
      .otherwise(pl.col('valid_concept_count'))
      .alias('valid_concept_count')
])

In [ ]:
schema_data['vocabulary_structure'].sort('valid_concept_count', descending=True).head()

In [ ]:
concept_domains_query = f"""
    SELECT
        domain_id,
        concept_class_id,
        vocabulary_id,
        COUNT(*) AS concept_count,
        COUNT(DISTINCT CASE WHEN standard_concept = 'S' THEN concept_id END) AS standard_count
    FROM `{dataset}.concept`
    WHERE domain_id IN (
        'Condition',
        'Drug',
        'Measurement',
        'Observation',
        'Procedure',
        'Visit',
        'Device'
    )
    GROUP BY domain_id, concept_class_id, vocabulary_id
    ORDER BY domain_id, concept_count DESC
    """

schema_data['concept_domains'] = polars_gbq(concept_domains_query)

In [ ]:
schema_data['concept_domains'] = schema_data['concept_domains'].with_columns([
    pl.when(pl.col('concept_count').is_between(1, 19))
      .then(-9)
      .otherwise(pl.col('concept_count'))
      .alias('concept_count'),
    
    pl.when(pl.col('standard_count').is_between(1, 19))
      .then(-9)
      .otherwise(pl.col('standard_count'))
      .alias('standard_count')
])

In [ ]:
schema_data['concept_domains'].head()

In [ ]:
measurement_concepts_query = f"""
    WITH measurement_counts AS (
        SELECT
            measurement_concept_id,
            COUNT(DISTINCT person_id) AS person_count,
            COUNT(*) AS measurement_count
        FROM `{dataset}.measurement`
        GROUP BY measurement_concept_id
        HAVING person_count > 100
    )
    SELECT
        mc.measurement_concept_id,
        c.concept_name,
        c.concept_code,
        c.vocabulary_id,
        c.concept_class_id,
        mc.person_count,
        mc.measurement_count
    FROM measurement_counts mc
    JOIN `{dataset}.concept` c
        ON mc.measurement_concept_id = c.concept_id
    ORDER BY mc.measurement_count DESC
    LIMIT 1000
    """

schema_data['measurement_concepts'] = polars_gbq(measurement_concepts_query)

In [ ]:
schema_data['measurement_concepts'].sort('measurement_count', descending=True).head()

In [ ]:
visit_concepts_query = f"""
    SELECT
        v.visit_concept_id,
        c.concept_name AS visit_type,
        COUNT(DISTINCT v.person_id) AS person_count,
        COUNT(*) AS visit_count
    FROM `{dataset}.visit_occurrence` v
    JOIN `{dataset}.concept` c
        ON v.visit_concept_id = c.concept_id
    GROUP BY v.visit_concept_id, c.concept_name
    ORDER BY visit_count DESC
    """

schema_data['visit_concepts'] = polars_gbq(visit_concepts_query)

In [ ]:
schema_data['visit_concepts'] = schema_data['visit_concepts'].with_columns([
    pl.when(pl.col('person_count').is_between(1, 19))
      .then(-9)
      .otherwise(pl.col('person_count'))
      .alias('person_count'),
    
    pl.when(pl.col('visit_count').is_between(1, 19))
      .then(-9)
      .otherwise(pl.col('visit_count'))
      .alias('visit_count')
])

In [ ]:
schema_data['visit_concepts'].sort('visit_count', descending=True).head()

In [ ]:
condition_vocabularies_query = f"""
    SELECT
        c.vocabulary_id,
        COUNT(DISTINCT co.person_id) AS person_count,
        COUNT(*) AS condition_count,
        COUNT(DISTINCT co.condition_concept_id) AS unique_concepts
    FROM `{dataset}.condition_occurrence` co
    JOIN `{dataset}.concept` c
        ON co.condition_source_concept_id = c.concept_id
    WHERE c.vocabulary_id IN ('ICD9CM', 'ICD10CM', 'SNOMED')
    GROUP BY c.vocabulary_id
    ORDER BY condition_count DESC
    """

schema_data['condition_vocabularies'] = polars_gbq(condition_vocabularies_query)

In [ ]:
schema_data['condition_vocabularies']

In [ ]:
for name, df in schema_data.items():
    gcs_path = f"{temp_bucket}/schema_data/{name}.tsv"
    df.write_csv(gcs_path, separator='\t')
    print(f"  Saved: {gcs_path} ({len(df):,} rows)")

In [ ]:
!mkdir -p schema_data

for name, df in schema_data.items():
    output_path = f"schema_data/{name}.tsv"
    df.write_csv(output_path, separator='\t')
    print(f"  Saved: {output_path} ({len(df):,} rows)")

## Provenance
We recommend reviewing the additional notebooks in the ```aou-tutorial-notebook``` bucket for more getting started resources.

In [ ]:
!date

In [ ]:
#Conda and pip installed packages:
!conda env export

In [ ]:
#JupyterLab extensions:
!jupyter labextension list

In [ ]:
#Number of cores:
!grep ^processor /proc/cpuinfo | wc -l

In [ ]:
#Memory:
!grep "^MemTotal:" /proc/meminfo

In [136]:
print_resource_usage()

Memory: 3.4GB / 202.8GB (1.7% used)
CPU: 0.0% used across 48 cores
LOW USAGE - Could downsize to save costs
